In [17]:
import pandas as pd
import re
from pathlib import Path


desktop_path = Path("/Users/compiling/Desktop/CS465_Data_Analysis/Destop_Experiment.txt")
vr_path = Path("/Users/compiling/Desktop/CS465_Data_Analysis/VR_Experiment.txt")

with open(desktop_path, 'r') as f:
    desktop_data = f.read()

with open(vr_path, 'r') as f:
    vr_data = f.read()


participant_blocks = {
    "Desktop": re.findall(r"PARTICPANT #[\s\S]*?(?=PARTICPANT|\Z)", desktop_data),
    "VR": re.findall(r"PARTICPANT #[\s\S]*?(?=PARTICPANT|\Z)", vr_data)
}


rows = []


def extract_participant_data(blocks, condition):
    for block in blocks:
        participant_match = re.search(r"PARTICPANT #(\d+(?:\.\d)?)", block)
        participant_id = int(float(participant_match.group(1))) if participant_match else -1
        tasks = re.findall(
            r"Task: (\d+)\nWPM: ([\d.]+)\nAccuracy: ([\d.]+)\nIncorrections: (\d+)\nTime: ([\d.]+)",
            block
        )
        for task_num, wpm, acc, errors, time in tasks:
            rows.append({
                "Participant": participant_id,
                "Task": int(task_num),
                "Condition": condition,
                "WPM": float(wpm),
                "Accuracy": float(acc),
                "Errors": int(errors),
                "Time": float(time)
            })


extract_participant_data(participant_blocks["Desktop"], "Desktop")
extract_participant_data(participant_blocks["VR"], "VR_Handless")  # Inverting as per user
extract_participant_data(participant_blocks["VR"], "VR_Hands")     # Inverting as per user


df = pd.DataFrame(rows)


participant_counts = df["Participant"].value_counts().sort_index()



In [18]:
print(df)

     Participant  Task Condition   WPM  Accuracy  Errors        Time
0              1     1   Desktop  81.0  0.989362       1   13.223040
1              1     2   Desktop  62.0  0.979798       2   17.193138
2              1     3   Desktop  43.0  0.935185       7   20.450686
3              1     4   Desktop  55.0  0.991453       1   24.575850
4              1     5   Desktop  48.0  0.983607       2   28.009271
..           ...   ...       ...   ...       ...     ...         ...
591           12     4  VR_Hands   0.0  0.318966      79  126.416637
592           12     5  VR_Hands   0.0  0.475410      64  179.858056
593           12     6  VR_Hands   0.0  0.125984     111   97.759663
594           12     7  VR_Hands   0.0  0.112782     118  151.102585
595           12     8  VR_Hands   0.0  0.210938     101  162.723556

[596 rows x 7 columns]


In [20]:
import pandas as pd


df = pd.read_csv("/Users/compiling/Desktop/CS465_Data_Analysis/Formatted_Experiment_Data_FIXED.txt", sep='\t')


df.rename(columns={"Participant": "Subject"}, inplace=True)


df = df[["Subject", "Task", "Condition", "WPM", "Accuracy", "Errors", "Time"]]


df.to_csv("/Users/compiling/Desktop/CS465_Data_Analysis/Formatted_Experiment_Data_FINAL.txt", sep='\t', index=False)


In [24]:



df = pd.read_csv("Formatted_Experiment_Data_FINAL.txt", sep="\t")


df = df.drop_duplicates(subset=["Subject", "Task", "Condition"])


target_conditions = ["Desktop", "VR_Hands", "VR_Handless"]
df = df[df["Condition"].isin(target_conditions)]


metrics = ["WPM", "Accuracy", "Errors", "Time"]

for metric in metrics:
    pivot = df.pivot_table(index='Subject', columns='Condition', values=metric, aggfunc='mean')
    pivot = pivot.dropna() 
    filename = f"friedman_{metric.lower()}.txt"
    pivot.to_csv(filename, sep="\t", index=False)
    print(f"Saved: {filename}")


Saved: friedman_wpm.txt
Saved: friedman_accuracy.txt
Saved: friedman_errors.txt
Saved: friedman_time.txt
